# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

# My Solution

Since the directions for this lab applied to a webpage version of IMDB that was out of date, I was directed by my instructor to conduct some webscraping on whatever piqued my interest, so long as I could learn or glean some helpful information from my work. I decided, therefore, to apply webscraping to see if a personal project of mine could be made easier and more effective. 

I've been trying to figure out a way to divide up the study of the different books of the Standard Works of the Church of Jesus Christ of Latter-day Saints into chunks of daily reading that are both manageable and consistent. I have tried doing succh breakdowns by verse, but have found that there is quite a wide variation in verse lengths, even in the same chapters or on the same pages. That inconsistency can lead to major differences in the amount of time a reader will spend on one chunk of daily reading. I often will read 17 verses one day and find it only spans about a page, and then read 17 verses the next day and find that the same number of verses spans two pages because the verses are longer on the second day than on the first. 

So, I wanted to use webscraping to see if I could more effectively breakup those daily chunks by the number of lines to read. This level of nuance will provide for a much higher level of consistency from day to day as I seek to come closer to Christ by studying in His word, while also helping me not be so stressed out when I find that one day's reading is twice as long as the day before's despite being the same number of verses.|

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re



I started off by setting my working directory and navigating to the Doctrine and Covenants webpage found on the main website of the Church, and set up a response request to start looking at the html code of the page. 

In [ ]:
os.chdir("D:\Faith and Religion Stuff\Come, Follow Me Breakdowns")
url = 'https://www.churchofjesuschrist.org/study/scriptures/dc-testament/dc/1?lang=eng'

response = requests.get(url)
response
response.content
response.headers
print(response.headers['Content-Type'])
soup = BeautifulSoup(response.content, "html.parser")
type(soup)
content = soup.prettify()  # This formats the HTML in a readable way
print(content)



Using beautiful soup allowed me to identify a few important pieces of information. First off, when looking at any given chapter of scripture, I found verse tags that gave me information about each verse and how it is displayed. Then, when looking at the contents page of the Doctrine and Covenants, I found that there were href links to each section, and in each of the < a > lines there was also a text title associated with the tag < a clclass="sc-omeqik-0 ewktus">. Having that information, I knew that I could get the information I needed. 

I got a lot of help from ChatGPT on this, but I was able to work with it download and access a google driver and use Selenium to do a lot of the same webscraping things that BeautifulSoup can do. 

After installing the driver and Selenium, I had to establish the path for accessing the driver, test to see if it was working, set options for Chrome, and create the service function I'd be using. 

In [ ]:
# Define the path to the chromedriver executable
chrome_driver_dir = r'C:\\Users\\bfran\\Downloads\\ChromeDriver\\chromedriver-win64'
chrome_driver_path = os.path.join(chrome_driver_dir, 'chromedriver.exe')

# Verify that the path is correct and the file exists
print(f"Checking if the chromedriver exists at: {chrome_driver_path}")
if not os.path.isfile(chrome_driver_path):
    raise FileNotFoundError(f"The chromedriver executable was not found at the specified path: {chrome_driver_path}")
else:
    print("Chromedriver found!")

# Set up the headless browser options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Set up the Chrome service
service = Service(chrome_driver_path)



From there, it was a lot of poking around and tweaking and adjusting. My first order of business was to see if I could even access the text of a verse. After that, I relied on ChatGPT's guidance to get the font-size and line heights to estimate the number of lines. 

In [ ]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Use driver to get url
driver.get(url)

# Allow some time for rendering
time.sleep(2)

# Find the element containing the text
container = driver.find_element(By.CSS_SELECTOR, '.verse')

# Get the text of the element
text = container.text

# Get the bounding rectangle of the element
rect = container.rect

# Print the information
print(f"Container dimensions: width={rect['width']} height={rect['height']}")
print(f"Text content: {text}")

# Check if the content height suggests line breaks
verse_element = driver.find_element(By.CSS_SELECTOR, '.verse')
font_size = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('font-size');", verse_element)

# Calculate line height
line_height_str = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('line-height');", verse_element)
line_height_numeric = int(re.search(r'\d+', line_height_str).group())  # Extract numeric value from string
num_lines = rect['height'] // line_height_numeric

print(f"Estimated number of lines: {num_lines}")

# Close the browser
driver.quit()
 


Once I established that I could do that with one verse, and that the estimate was accurate, I could do it for a whole chapter.

In [ ]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Run the driver
driver.get(url)

# Allow some time for rendering
time.sleep(2)

# Find all elements containing the text
verses = driver.find_elements(By.CSS_SELECTOR, '.verse')

# Iterate over each verse element
for verse in verses:
    # Get the text of the element
    text = verse.text

    # Get the bounding rectangle of the element
    rect = verse.rect

    # Print the information
    print(f"Container dimensions: width={rect['width']} height={rect['height']}")
    print(f"Text content: {text}")

    # Calculate font size
    font_size = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('font-size');", verse)

    # Calculate line height
    line_height_str = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('line-height');", verse)
    line_height_numeric = int(re.search(r'\d+', line_height_str).group())  # Extract numeric value from string

    # Calculate number of lines
    num_lines = rect['height'] // line_height_numeric

    print(f"Estimated number of lines: {num_lines}")
    print("-" * 50)  # Separator for clarity

# Close the browser
driver.quit()



After getting that to work, I knew I could move on to just saving the information collected to a dataframe. 

In [ ]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Run the driver
driver.get(url)

# Find all elements containing the text
verses = driver.find_elements(By.CSS_SELECTOR, '.verse')

# Initialize a list to store data dictionaries
data_list = []

# Iterate over each verse element
for verse in verses:
    # Get the text of the element
    text = verse.text

    # Extract verse number (assuming it's in the format "1 ", "2 ", etc.)
    verse_number = text.split(' ')[0]  # Assuming verse number is at the start of text
    
    # Get the bounding rectangle of the element
    rect = verse.rect

    # Calculate line height
    line_height_str = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('line-height');", verse)
    line_height_numeric = int(re.search(r'\d+', line_height_str).group())  # Extract numeric value from string

    # Calculate number of lines
    num_lines = rect['height'] // line_height_numeric

    # Append data dictionary to list
    data_list.append({
        'Verse Number': verse_number,
        'Num Lines': num_lines
    })

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data_list)

# Print the DataFrame (optional)
print(df)

# Close the browser
driver.quit()


And once that was working, I knew I could just write a function that would take the input of a url to a chapter of scripture and then use that url to get the line lengths of each verse. 

In [ ]:
def get_verse_lines(url):
    """
    This function takes a URL as input and returns the number of lines in the HTML content of all the verses in a given chapter of Holy Scripture and stores the verses and line counts in a pandas dataframe.
    """
    # Import needed packages
    import pandas as pd
    import requests
    import os
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    import re

    # Define the path to the chromedriver executable
    chrome_driver_dir = r'C:\\Users\\bfran\\Downloads\\ChromeDriver\\chromedriver-win64'
    chrome_driver_path = os.path.join(chrome_driver_dir, 'chromedriver.exe')

    # Set up the headless browser options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")

    # Set up the Chrome service
    service = Service(chrome_driver_path)    
    
    # Initialize the Chrome WebDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Run the driver
    driver.get(url)

    # Find all elements containing the text
    verses = driver.find_elements(By.CSS_SELECTOR, '.verse')

    # Initialize a list to store data dictionaries
    data_list = []

    # Iterate over each verse element
    for verse in verses:
        # Get the text of the element
        text = verse.text

        # Extract verse number (assuming it's in the format "1 ", "2 ", etc.)
        verse_number = text.split(' ')[0]  # Assuming verse number is at the start of text
    
        # Get the bounding rectangle of the element
        rect = verse.rect

        # Calculate line height
        line_height_str = driver.execute_script("return window.getComputedStyle(arguments[0]).getPropertyValue('line-height');", verse)
        line_height_numeric = int(re.search(r'\d+', line_height_str).group())  # Extract numeric value from string

        # Calculate number of lines
        num_lines = rect['height'] // line_height_numeric

        # Append data dictionary to list
        data_list.append({
            'Verse Number': verse_number,
            'Num Lines': num_lines
        })

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data_list)

    # Close the browser
    driver.quit()

    return df



Having successfully done that, I could have called it a day, but instead I decided that if I could get that information, surely I could also use webscraping to get a dataframe of links that I could feed into that function using an iteration loop. 

Again, I relied on ChatGPT to walk me through the process and help me gather the right information. 

I started with just getting the links. 

In [ ]:
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Navigate to the page with your elements
    driver.get('https://www.churchofjesuschrist.org/study/scriptures/dc-testament?lang=eng')

    # Find all elements with the specified class name
    elements = driver.find_elements(By.CLASS_NAME, 'sc-omeqik-0')

    # Initialize a list to store href values
    href_list = []
    title_list = []

    # Iterate over each element and extract the href attribute
    for element in elements:
        href = element.get_attribute('href')
        href_list.append(href)

    # Create a DataFrame to store the href values
    urls_df = pd.DataFrame({'Href': href_list})

    # Print the DataFrame (optional)
    print(urls_df)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the browser
    driver.quit()



But the links would do no good at all if I didn't have the section titles, because I realized I'd also want to save each section or chapter's verse lengths as separate dataframes that I could export as csv files. So, I spent a long time trying to get that worked out. Well, actually, I spent a long time trying to work out nonexistent issues because I didn't realize the output of my attempts to get titles and links was a scrollable cell, so I thought I kept just getting links. 

Some additional things added to this cell's code are things that I had to remove - there were more titles than href links, so I had to go in and remove the titles that didn't have links. Additionally, Official Declaration 1 and 2 have different links, but the same title, so I had to duplicate that title. This process was just a whole lot of trial and error. 

In [ ]:
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Navigate to the page with your elements
    driver.get('https://www.churchofjesuschrist.org/study/scriptures/dc-testament?lang=eng')

    # Find all elements with the specified classes
    href_elements = driver.find_elements(By.CSS_SELECTOR, 'a.sc-omeqik-0.ewktus')
    title_elements = driver.find_elements(By.CSS_SELECTOR, 'p.title')

    # Debugging: Print lengths of elements found
    print(f"Number of href elements: {len(href_elements)}")
    print(f"Number of title elements: {len(title_elements)}")

    # Skip the very first title element
    if title_elements:
        title_elements = title_elements[1:145]

    # Delete the 4th title element (index 3)
    if len(title_elements) > 3:
        del title_elements[3]

    # Duplicate the last title element
    if title_elements:
        title_elements.append(title_elements[-1])

    # Delete the 4th title element (index 3)
    if len(href_elements) > 142:
        del href_elements[142]

    # Initialize lists to store href and title values
    href_list = [element.get_attribute('href') for element in href_elements]
    title_list = [element.text for element in title_elements]

    # Create a list of dictionaries to store matched data
    matched_data = []
    min_length = min(len(href_list), len(title_list))

    # Match hrefs and titles based on the minimum length
    for i in range(min_length):
        matched_data.append({'Title': title_list[i], 'Href': href_list[i]})

    # Create a DataFrame from matched data
    dc_links_df = pd.DataFrame(matched_data)

    # Set Pandas display options to show full content of 'Href' column
    pd.set_option('display.max_colwidth', None)

    # Print the DataFrame to verify the 'Href' column contents
    print(dc_links_df)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the browser
    driver.quit()



Once it was working, though, I could subset the dataframe into the introductory stuff, the actual sections of the Doctrine and Covenants, and the Official Declarations. I had to do this because, while the sections of the Doctrine and Covenants are "versified," the introductory materials and Official Declarations are not, so I'll have to get those line lengths later. 

In [ ]:
dc_sections = dc_links_df.iloc[4:142]
dc_intro = dc_links_df.iloc[:3]
dc_ods = dc_links_df[142:]

After subsetting the data all I had to do was establish my iteration loop, and the result was a nice set of individual csv files that I'll be able to use SQL queries to join and access so that I can make my own daily breakdowns later. 

In [ ]:
# Define the directory path where you want to save the CSV files
dir_path = r'D:\\Faith and Religion Stuff\\Come, Follow Me Breakdowns\\D&C Sections Verse Lines'

# Ensure the directory exists, if not create it
os.makedirs(dir_path, exist_ok=True)

for index, row in dc_sections.iterrows():
    title = row['Title']
    link = row['Href']

    verse_lines_df = get_verse_lines(link)

    if verse_lines_df is not None and not verse_lines_df.empty:
        csv_filename = f'{title.replace(" ","_").lower()}_verse_lines.csv'
        full_path = os.path.join(dir_path, csv_filename)
        
        # Debugging: Print full path to ensure it's correct
        print(f'Saving to: {full_path}')

        verse_lines_df.to_csv(full_path, index=False)

        print(f'CSV file for "{title}" saved successfully as {csv_filename}.')
    else:
        print(f'No data for "{title}", skipping CSV creation.')

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here